In [6]:
import matplotlib.pyplot as plt
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

import numpy as np
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
input_shape = (32, 32, 1)

x_train shape: (60000, 28, 28, 1)
Number of images in x_train 60000
Number of images in x_test 10000


In [7]:
#Lenet 5

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
model = tf.keras.models.Sequential([

tf.keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)),
tf.keras.layers.AveragePooling2D(),

tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
tf.keras.layers.AveragePooling2D(),

tf.keras.layers.Flatten(),

tf.keras.layers.Dense(units=120, activation='relu'),

tf.keras.layers.Dense(units=84, activation='relu'),

tf.keras.layers.Dense(units=10, activation = 'softmax')
]);

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 6)         60        
_________________________________________________________________
average_pooling2d_2 (Average (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
average_pooling2d_3 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               69240     
_________________________________________________________________
dense_4 (Dense)              (None, 84)               

In [3]:
# implementing a callback function to terminate training once training reaches 98% accuracy for validation data

validation_accuracy = 1

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("val_accuracy") is not None and logs.get("val_accuracy") >= validation_accuracy):
            print("\nReached desired validation accuracy, so cancelling training")
            self.model.stop_training=True
            
callbacks = myCallback()

In [4]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [5]:
with tf.device('/gpu:0'):
    my_model = model.fit(x=x_train,y=y_train, epochs=10, validation_data=(x_test, y_test), verbose=1, 
                                  callbacks = [callbacks], workers=10)

Epoch 1/10
1875/1875 [==============================] - 18s 8ms/step - loss: 0.2260 - accuracy: 0.9322 - val_loss: 0.0938 - val_accuracy: 0.9717
Epoch 2/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0744 - accuracy: 0.9772 - val_loss: 0.0535 - val_accuracy: 0.9831
Epoch 3/10
1875/1875 [==============================] - 20s 10ms/step - loss: 0.0539 - accuracy: 0.9831 - val_loss: 0.0445 - val_accuracy: 0.9860
Epoch 4/10
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0419 - accuracy: 0.9865 - val_loss: 0.0395 - val_accuracy: 0.9877
Epoch 5/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0346 - accuracy: 0.9889 - val_loss: 0.0321 - val_accuracy: 0.9898
Epoch 6/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0287 - accuracy: 0.9911 - val_loss: 0.0369 - val_accuracy: 0.9873
Epoch 7/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0234 - accuracy: 0.9923 - val_loss: 0.0305 - val_a

In [ ]:
filename = "model5"
filepath = r"model5"
tf.keras.models.save_model(
    model,
    filepath,
    overwrite=True,
    include_optimizer=True,
    save_format="tf",
    signatures=None
)

model.save(filename+".h5")

# Convert the model.
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(filename+'.h5')
tflite_model = converter.convert()

# Save the model.
with open(filename+'.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
  historyFileName = "hist"
  with open(historyFileName+".txt",'w') as f:
    for k in my_model.history.keys():
        print(k,file=f)
        for i in my_model.history[k]:
            print(i,file=f)

In [ ]:
acc = my_model.history['accuracy'] #training accuracy scores from the model that has been trained
val_acc = my_model.history['val_accuracy'] #validation accuracy scores from the model that has been trained
loss = my_model.history['loss'] #training loss scores from the model that has been trained
val_loss = my_model.history['val_loss'] #validation loss scores from the model that has been trained

epochs = range(len(acc)) #x axis

plt.plot(epochs, acc, 'bo', label = 'Dokładność treningu')
plt.plot(epochs, val_acc, 'b', label = 'Dokładność walidacji')
plt.title('Dokładność treningu i walidacji')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'ro', label = 'Błąd przewidywania treningu')
plt.plot(epochs, val_loss, 'r', label = 'Błąd przewidywania walidacji')
plt.title('Błąd przewidywania treningu i walidacji')

plt.legend()

plt.show()